In [ ]:
def backpropagation_3b2b(z2, a2, z3, a3, w3, z4, a4, w4, labels, daten):
    """
    Backpropagation basierend auf den expliziten Formeln von 3Blue1Brown.
    Nutzt die Kettenregel: dC/dw = dC/da * da/dz * dz/dw
    """
    # Anzahl der Trainingsbeispiele (n) für die Durchschnittsbildung
    n = labels.size
    
    # Der gewünschte Ziel-Vektor (y)
    y = eins_aus_n(labels)

    # --- SCHICHT 3 (Ausgabeschicht) ---
    # 1. Ableitung der Kostenfunktion nach der Aktivierung: 2 * (aL - y) 
    dC_da4 = 2 * (a4 - y)
    
    # 2. Ableitung der Aktivierung nach der gewichteten Summe: sigma'(zL)
    da4_dz4 = ableitung_sigmoid(z4)
    
    # Zusammengefasst (Fehler der Schicht 3): dC/dz4
    dz4 = dC_da4 * da4_dz4 

    # Gradienten für Gewichte und Biases der Schicht 3 
    # dw4 = dz4 * a3 (Durchschnitt über alle Beispiele n)
    dw4 = (1/n) * dz4.dot(a3.T)
    db4 = (1/n) * np.sum(dz4, axis=1, keepdims=True)

    # --- SCHICHT 2 (Versteckte Schicht) ---
    # 1. Einfluss der Aktivierung a3 auf die Kosten (über alle Pfade j summiert)
    # dC/da3 = w4.T * dz4
    dC_da3 = w4.T.dot(dz4)
    
    # 2. Fehler der Schicht 2: dC/da3 * sigma'(z3)
    dz3 = dC_da3 * ableitung_sigmoid(z3)

    dw3 = (1/n) * dz3.dot(a2.T)
    db3 = (1/n) * np.sum(dz3, axis=1, keepdims=True)

    # --- SCHICHT 1 (Erste versteckte Schicht) ---
    # Analog zur Schicht 2 [11, 12]
    dC_da2 = w3.T.dot(dz3)
    dz2 = dC_da2 * ableitung_sigmoid(z2)

    dw2 = (1/n) * dz2.dot(daten.T)
    db2 = (1/n) * np.sum(dz2, axis=1, keepdims=True)

    return dw2, db2, dw3, db3, dw4, db4

Die grünen und orangen Kommentare sind von Google NotebookLM.   
  
Die Ableitungen werden Schritt für Schritt wie in der ABA ausgeführt. Außerdem wird hier nicht das Matrizenprodukt verwendet, sondern die Matrizen werden elementweise multipliziert.  
  
Der erste entscheidende Unterschied ist die Ableitung nach a4, anschließend wird dies mit der Ableitung der Aktivierungsfunktion der gewichteten Summe multipliziert.    
  
Um nun die Änderungen der Gewichte auszurechnen, wird dies gemäß der Formel aus der ABA mit den Aktivierungen der vorherigen Schicht multipliziert (diesmal Matrizenmultiplikation) und durch die Anzahl der Trainingsbeispiele dividiert, da diese zuvor in der Matrizenmultiplikation aufsummiert wurden.  
    
Um nun nach a3 abzuleiten, werden wie im anderen Code die Gewichtsmatrizen transponiert und mit der gewichtenten Summe der letzten Schicht Matrix-multipliziert. Dies wird für alle Schichten gemacht, bis die Änderungen für alle Gewichts- und Biasmatrizen bekannt sind. Diese werden dann wie im anderen Code geändert.

In [ ]:
def gradientenabstieg(daten,label,wiederholungen,lernrate):
    w1,b1,w2,b2,w3,b3=matrizen_erstellen()
    for i in range(wiederholungen):
        z1,a1,z2,a2,z3,a3=Abfrage(w1,b1,w2,b2,w3,b3, daten)
        dw1,db1,dw2,db2,dw3,db3=backpropagation(z1,a1,z2,a2,w2,z3,a3,w3,label,daten, )
        w1,b1,w2,b2,w3,b3=parameter_aktualisieren(w1,b1,w2,b2,w3,b3,dw1,db1,dw2,db2,dw3,db3,lernrate)
        if i%100==0:
            print("Wiederholungen: ",i)
            print("Accuracy: ", wie_akkurat(vorhersage_machen(a3),label))
    return w1,b1,w2,b2,w3,b3,l